Here we show how you can replace the IDT in a running Linux system with a new one.

The steps are as follows:
1. Find out where (in virtual address space) the system IDT is located.
2. Allocate a page of memory in the kernel address space.
3. Copy the existing idt onto the new page
4. "Install" the new idt by updating the idtr register.

The new idt is now in place, but the old one is still in the kernel address space. We could free it if we wanted to, but instead, we switch back onto it. This demonstrates a workflow that encourages restoring the system as expected when the modification is no longer needed.

In [7]:
import sys
sys.path.append("../common_code/")
import common as com


In [8]:
import os
# Get full path to idt_tool executable
path = os.path.abspath(com.symbios_path + "Tools/bin")
os.environ["PATH"] += ":" + path

path = os.path.abspath(com.symbios_path + "Tools/bin/recipes")
os.environ["PATH"] += ":" + path

In [9]:
# Read the old IDT into a dataframe
old_idt = com.read_whole_idt()

In [10]:
ret = com.run_cmd("taskset -c 0 idt_tool -g")
addr_old_idt = ret.stdout.splitlines()[0]
# Prepend 0x if not there.
addr_old_idt = hex(int(addr_old_idt, 16))
# Print the old IDT is located at this address

print("Old IDT is located at: ", addr_old_idt)

Old IDT is located at:  0xffffc9000880f000


In [11]:
# Allocate a kernel page, copy the old idt onto it, and return the address of this page
ret = com.run_cmd("taskset -c 0 idt_tool -c")
addr_new_idt = ret.stdout.splitlines()[0]
addr_new_idt = hex(int(addr_new_idt, 16))

print("New IDT is located at: ", addr_new_idt)

New IDT is located at:  0xffffc90003f57000


In [13]:
# Install the new IDT
com.run_cmd("taskset -c 0 idt_tool -i -a " + addr_new_idt)

# Get the currently loaded IDT ptr
ret = com.run_cmd("taskset -c 0 idt_tool -g")
addr_current_idt = ret.stdout.splitlines()[0]
addr_current_idt = hex(int(addr_current_idt, 16))

print("Current IDT is located at: ", addr_current_idt)

Current IDT is located at:  0xffffc90003f57000


In [14]:
# Now reinstall the origional IDT
com.run_cmd("taskset -c 0 idt_tool -i -a " + addr_old_idt)

# Get the currently loaded IDT ptr
ret = com.run_cmd("taskset -c 0 idt_tool -g")
addr_current_idt = ret.stdout.splitlines()[0]
addr_current_idt = hex(int(addr_current_idt, 16))

print("Current IDT is located at: ", addr_current_idt)

Current IDT is located at:  0xffffc9000880f000
